In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [28]:
import osmnx as ox
import pandas as pd
import numpy as np
from Data import Data 
import networkx as nx
from networkx import algorithms 
import math

In [4]:
graph = ox.graph_from_address('Athens, Municipality of Athens, Regional Unit of Central Athens, Attica, 10667, Greece', network_type='drive') 

In [5]:
df = pd.read_pickle("Data_sample.pkl")
X = df.copy()

In [76]:
#get the nearest edge and node for each location:
nodes = np.zeros(len(df))
EdgeStart = np.zeros(len(df))
EdgeEnd = np.zeros(len(df))
for i in range(len(df)):
    tup1 = (df.iloc[i]['lat'], df.iloc[i]['lon'])
    nodes[i] = ox.get_nearest_node(graph, tup1 , method='euclidean')
    start, end, c = ox.get_nearest_edge(graph, tup1)
    EdgeStart[i] = start
    EdgeEnd[i] = end

In [78]:
df['Nearest Node'] = nodes
df['Nearest Edge Start'] =  EdgeStart
df['Nearest Edge End'] =  EdgeEnd

In [79]:
df

lat        lon    speed  lon_acc  lat_acc  type  traveled_d  \
id time                                                                       
1  0.00   37.977482  23.735405  34.0206  -0.0167   0.0074  Taxi      320.26   
   0.04   37.977485  23.735402  34.0197   0.0040   0.0086  Taxi      320.26   
   0.08   37.977488  23.735400  34.0211   0.0167   0.0097  Taxi      320.26   
   0.12   37.977491  23.735398  34.0245   0.0294   0.0108  Taxi      320.26   
   0.16   37.977494  23.735395  34.0305   0.0540   0.0120  Taxi      320.26   
...             ...        ...      ...      ...      ...   ...         ...   
5  29.24  37.979836  23.733313  36.8072  -3.7893   1.1652   Car      272.25   
   29.28  37.979838  23.733311  36.4768  -3.8277   1.1690   Car      272.25   
   29.32  37.979841  23.733309  36.1375  -3.8118   1.1665   Car      272.25   
   29.36  37.979844  23.733306  35.7859  -3.8818   1.1568   Car      272.25   
   29.40  37.979847  23.733304  35.4092  -4.0068   1.1387   Car      272.25   

          avg_speed  Nearest Node  Nearest Edge First NodeID  \
id time                                                        
1  0.00   35.366264  3.339822e+09                  250692418   
   0.04   35.366264  3.339822e+09                  250692418   
   0.08   35.366264  3.339822e+09                  250692418   
   0.12   35.366264  3.339822e+09                  250692418   
   0.16   35.366264  3.339822e+09                  250692418   
...             ...           ...                        ...   
5  29.24  33.336716  2.506989e+08                  250692418   
   29.28  33.336716  2.506989e+08                  250692418   
   29.32  33.336716  2.506989e+08                  250692418   
   29.36  33.336716  2.506989e+08                  250692418   
   29.40  33.336716  2.506989e+08                  250692418   

          Nearest Edge Second NodeID  Nearest Edge Start  Nearest Edge End  
id time                                                                     
1  0.00                    250692401        3.339822e+09       250691723.0  
   0.04                    250692401        3.339822e+09       250691723.0  
   0.08                    250692401        3.339822e+09       250691723.0  
   0.12                    250692401        3.339822e+09       250691723.0  
   0.16                    250692401        3.339822e+09       250691723.0  
...                              ...                 ...               ...  
5  29.24                   250692401        2.506989e+08       250698925.0  
   29.28                   250692401        2.506989e+08       250698925.0  
   29.32                   250692401        2.506989e+08       250698925.0  
   29.36                   250692401        2.506989e+08       250698925.0  
   29.40                   250692401        2.506989e+08       250698925.0  

[3460 rows x 13 columns]

In [80]:
def Distance_achieved(edge_node1,edge_node2,lonV, latV):
    #assuming that edge_node1 is the starting point
    lon0 = graph.nodes[edge_node1]['x']
    lon1 = graph.nodes[edge_node1]['x']
    lat0 = graph.nodes[edge_node1]['y']
    lat1 = graph.nodes[edge_node2]['y']
    a = Distance_Euc(lon0, lat0, lon1, lat1)
    b = Distance_Euc(lon0, lat0, lonV, latV)
    return b/a

In [81]:
def Distance_Euc(lon0, lat0, lon1, lat1):
    lon0 = math.radians(lon0)
    lat0 = math.radians(lat0)
    lon1 = math.radians(lon1)
    lat1 = math.radians(lat1)
    dlon = lon1 - lon0
    dlat = lat1 - lat0
    a = math.sin(dlat / 2)**2 + math.cos(lat0) * math.cos(lat1) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return 6373.0*c

In [82]:
Dis = np.zeros(len(df))
for i in range(len(df)):
    b = df.iloc[i]['lon']
    c = df.iloc[i]['lat']
    a = Distance_achieved(df.iloc[i]['Nearest Edge Start'], df.iloc[i]['Nearest Edge End'], b, c)
    Dis[i] = a

In [84]:
df['Distance Achieved'] = Dis

In [85]:
df

lat        lon    speed  lon_acc  lat_acc  type  traveled_d  \
id time                                                                       
1  0.00   37.977482  23.735405  34.0206  -0.0167   0.0074  Taxi      320.26   
   0.04   37.977485  23.735402  34.0197   0.0040   0.0086  Taxi      320.26   
   0.08   37.977488  23.735400  34.0211   0.0167   0.0097  Taxi      320.26   
   0.12   37.977491  23.735398  34.0245   0.0294   0.0108  Taxi      320.26   
   0.16   37.977494  23.735395  34.0305   0.0540   0.0120  Taxi      320.26   
...             ...        ...      ...      ...      ...   ...         ...   
5  29.24  37.979836  23.733313  36.8072  -3.7893   1.1652   Car      272.25   
   29.28  37.979838  23.733311  36.4768  -3.8277   1.1690   Car      272.25   
   29.32  37.979841  23.733309  36.1375  -3.8118   1.1665   Car      272.25   
   29.36  37.979844  23.733306  35.7859  -3.8818   1.1568   Car      272.25   
   29.40  37.979847  23.733304  35.4092  -4.0068   1.1387   Car      272.25   

          avg_speed  Nearest Node  Nearest Edge First NodeID  \
id time                                                        
1  0.00   35.366264  3.339822e+09                  250692418   
   0.04   35.366264  3.339822e+09                  250692418   
   0.08   35.366264  3.339822e+09                  250692418   
   0.12   35.366264  3.339822e+09                  250692418   
   0.16   35.366264  3.339822e+09                  250692418   
...             ...           ...                        ...   
5  29.24  33.336716  2.506989e+08                  250692418   
   29.28  33.336716  2.506989e+08                  250692418   
   29.32  33.336716  2.506989e+08                  250692418   
   29.36  33.336716  2.506989e+08                  250692418   
   29.40  33.336716  2.506989e+08                  250692418   

          Nearest Edge Second NodeID  Nearest Edge Start  Nearest Edge End  \
id time                                                                      
1  0.00                    250692401        3.339822e+09       250691723.0   
   0.04                    250692401        3.339822e+09       250691723.0   
   0.08                    250692401        3.339822e+09       250691723.0   
   0.12                    250692401        3.339822e+09       250691723.0   
   0.16                    250692401        3.339822e+09       250691723.0   
...                              ...                 ...               ...   
5  29.24                   250692401        2.506989e+08       250698925.0   
   29.28                   250692401        2.506989e+08       250698925.0   
   29.32                   250692401        2.506989e+08       250698925.0   
   29.36                   250692401        2.506989e+08       250698925.0   
   29.40                   250692401        2.506989e+08       250698925.0   

          Distance Achieved  
id time                      
1  0.00            0.592525  
   0.04            0.595703  
   0.08            0.598527  
   0.12            0.601351  
   0.16            0.604529  
...                     ...  
5  29.24           0.617300  
   29.28           0.619753  
   29.32           0.622959  
   29.36           0.626638  
   29.40           0.629844  

[3460 rows x 14 columns]

38.295199201188716

0.31964235624777326

0.373240968109744

0.05806963530107411

(3339821648,
 250691723,
 0,
 2.0233523004191226e-05)

3339821648

23.7358882

(3339821648, 250691723, 0)

0.0788241591869522

0.08177279461877263

0.16054182547279708

3339821648

In [77]:
nodes

array([3.33982165e+09, 3.33982165e+09, 3.33982165e+09, ...,
       2.50698925e+08, 2.50698925e+08, 2.50698925e+08])